In [1]:
import json
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer


In [2]:
import ssl
import os

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
nltk.data.path.append('/tmp/nltk_data')
nltk.download("punkt", download_dir='/tmp/nltk_data')
nltk.download("stopwords", download_dir='/tmp/nltk_data')
nltk.download("wordnet", download_dir='/tmp/nltk_data')



[nltk_data] Downloading package punkt to /tmp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Directories
RAW_DATA_DIR = "/Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Raw"
PROCESSED_DATA_DIR = "/Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed"

In [7]:
# Ensure processed directory exists
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Load ColBERT tokenizer (for FAISS processing)
colbert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Stopwords for optional removal
STOPWORDS = set(stopwords.words("english"))

# Lemmatizer
lemmatizer = WordNetLemmatizer()

In [8]:
def clean_text(text):
    """Basic text cleaning: remove URLs, special characters, and extra spaces."""
    if not text:
        return ""
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

In [9]:
def chunk_text(text, tokenizer, max_length=512, stride=256):
    """Splits text into overlapping chunks to fit within model limits."""
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i: i + max_length] for i in range(0, len(tokens), stride)]
    return [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]

In [10]:
### 1️⃣ Preprocessing for Database (BM25 + FAISS) ###
def preprocess_for_db():
    """Preprocesses data for BM25 (Elasticsearch) and FAISS (ColBERT)."""
    for filename in os.listdir(RAW_DATA_DIR):
        if filename.endswith("_comments.json") or filename.endswith("_submissions.json"):
            input_file = os.path.join(RAW_DATA_DIR, filename)
            output_file = os.path.join(PROCESSED_DATA_DIR, filename.replace(".json", "_cleaned.json"))
            
            with open(input_file, "r") as infile, open(output_file, "w") as outfile:
                for line in infile:
                    data = json.loads(line)
                    
                    # Process body or selftext depending on file type
                    text_field = "selftext" if "selftext" in data else "body"
                    data[text_field] = clean_text(data.get(text_field, ""))
                    
                    # FAISS-specific: Chunk tokenized text for ColBERT
                    data["tokenized_body"] = chunk_text(data[text_field], colbert_tokenizer)
                    
                    json.dump(data, outfile)
                    outfile.write("\n")
            print(f"Database preprocessing complete: {output_file}")

In [ ]:
### 1️⃣ Preprocessing for Database (BM25 + FAISS) ###
def preprocess_for_db():
    """Preprocesses data for BM25 (Elasticsearch) and FAISS (ColBERT)."""
    for filename in os.listdir(RAW_DATA_DIR):
        if filename.endswith("_comments.json") or filename.endswith("_submissions.json"):
            input_file = os.path.join(RAW_DATA_DIR, filename)
            output_file = os.path.join(PROCESSED_DATA_DIR, filename.replace(".json", "_cleaned.json"))
            
            with open(input_file, "r") as infile, open(output_file, "w") as outfile:
                for line in infile:
                    data = json.loads(line)
                    
                    # Process body or selftext depending on file type
                    text_field = "selftext" if "selftext" in data else "body"
                    data[text_field] = clean_text(data.get(text_field, ""))
                    
                    # FAISS-specific: tokenize for ColBERT
                    data["tokenized_body"] = colbert_tokenizer.tokenize(data[text_field])
                    
                    json.dump(data, outfile)
                    outfile.write("\n")
            print(f"Database preprocessing complete: {output_file}")

In [ ]:
# Run preprocessing
if __name__ == "__main__":
    preprocess_for_db()

Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x104018b60>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/artificial_comments_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/artificialinteligence_comments_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/openai_comments_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/artificialinteligence_submissions_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/chatgpt_submissions_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/transhumanism_submissions_cleaned.json
Database preprocessing complete: /Users/angjelin/Library/CloudStorage/Box-Box/Reddit Vector DB/Data/Processed/chatg

: 